# 🦈 SHARK CRYPTO ANALYZER - ALL-IN-ONE (2026)
Hệ thống tự động phát hiện và lọc tin tức thị trường Crypto.

**Hướng dẫn:** Nhấn `Runtime` -> `Run all` để chạy toàn bộ.

In [ ]:
# @title 1. Cài đặt thư viện và Cấu hình
!pip install google-genai feedparser playwright pydantic aiohttp
!playwright install chromium
!playwright install-deps

import json
import os
import ssl
import asyncio
import time
import yaml
import feedparser
import aiohttp
import random
from datetime import datetime
from google import genai
from google.genai import types, errors
from pydantic import BaseModel
from playwright.async_api import async_playwright

# Mã màu ANSI (Đã tối ưu cho nền sáng Colab)
BLUE = "\033[34m"
CYAN = "\033[36m"
GREEN = "\033[32m"
YELLOW = "\033[33m"
RED = "\033[31m"
BOLD = "\033[1m"
RESET = "\033[0m"

# Danh sách cấu hình giả lập thiết bị (User-Agent + Viewport)
THIET_BI_GIA_LAP = [
    {"ua": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36", "vp": {"width": 1920, "height": 1080}, "is_mobile": False},
    {"ua": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36", "vp": {"width": 1440, "height": 900}, "is_mobile": False},
    {"ua": "Mozilla/5.0 (iPhone; CPU iPhone OS 17_2_1 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.2 Mobile/15E148 Safari/604.1", "vp": {"width": 390, "height": 844}, "is_mobile": True},
    {"ua": "Mozilla/5.0 (Linux; Android 13; SM-S918B) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Mobile Safari/537.36", "vp": {"width": 360, "height": 800}, "is_mobile": True},
    {"ua": "Mozilla/5.0 (iPad; CPU OS 17_0 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.0 Mobile/15E148 Safari/604.1", "vp": {"width": 820, "height": 1180}, "is_mobile": True}
]

# --- CẤU HÌNH TẠI ĐÂY ---
GOOGLE_API_KEY = "NHAP_KEY_O_DAY" # @param {type:"string"}
SHARK_CONTEXT = "BẠN LÀ MÁY LỌC TIN CRYPTO THUẦN TÚY. Chỉ chấp nhận các tin tức trực tiếp về: Bitcoin, Ethereum, Altcoins, Blockchain, DeFi, NFT, sàn giao dịch tiền số. LOẠI BỎ TẤT CẢ các tin về: Chứng khoán truyền thống, Giá Vàng, Ngân hàng trung ương, Bất động sản, hoặc Kinh tế vĩ mô chung TRỪ KHI tin đó nói trực tiếp về ảnh hưởng tới Crypto. Nếu không chắc chắn, hãy loại bỏ. Mục tiêu hàng đầu là thu thập các tin tức quan trọng về thị trường CRYPTO." # @param {type:"string"}
LIMIT_DAYS = 1 # @param {type:"slider", min:1, max:30, step:1}
CRAWL_CONCURRENCY = 3 # @param {type:"slider", min:1, max:5, step:1}

# @markdown Nhập các link RSS cách nhau bởi dấu phẩy hoặc xuống dòng:
RSS_LINKS_RAW = "http://theeconomist.tumblr.com/rss, https://99bitcoins.com/feed, https://ambcrypto.com/feed, https://asia.nikkei.com/rss/feed/nar, https://beincrypto.com/feed, https://bitcoinik.com/feed, https://bitcoinist.com/feed, https://bitcoinmagazine.com/feed, https://bitcoinvn.io/news/feed, https://blockchain.news/rss, https://blockonomi.com/feed, https://blockworks.co/feed, https://blog.bitfinex.com/feed, https://blog.bitmex.com/feed, https://blogtienao.com/feed, https://ca.finance.yahoo.com/news/rss/cdn_mkts, https://coin68.com/rss/trang-chu.rss, https://coincheckup.com/blog/feed, https://coingape.com/feed, https://coingeek.com/feed, https://coinidol.com/rss2, https://coinjournal.net/feed, https://coinmetrics.substack.com/feed, https://coinpedia.org/feed, https://coinspeaker.com/feed, https://cointelegraph.com/rss, https://consensys.io/blog/rss.xml, https://crypto-news-flash.com/feed, https://crypto-news.net/feed, https://crypto.news/feed, https://cryptobriefing.com/feed, https://cryptodaily.co.uk/feed, https://cryptonews.com/news/feed, https://cryptopanic.com/news/rss, https://cryptopotato.com/feed, https://cryptoslate.com/feed, https://cryptoverze.com/feed, https://dailyhodl.com/bitcoin-news/feed, https://dcforecasts.com/feed, https://decrypt.co/feed, https://e-cryptonews.com/feed, https://feeder.co/discover/c4c71588e6/decrypt-co-utm_source-feed-utm_medium-feed, https://feeds.bloomberg.com/crypto/news.rss, https://finance.yahoo.com/news/rssindex, https://flipboard.com/@WSJ.rss, https://insidebitcoins.com/feed, https://insights.glassnode.com/rss, https://kriptonovini.bg/rss.xml, https://livebitcoinnews.com/feed, https://news.bitcoin.com/feed, https://news.google.com/atom/topics/CAAqJAgKIh5DQkFTRUFvS0wyMHZNSFp3YWpSZlloSUNaVzRvQUFQAQ?hl=en-US&gl=US&ceid=US%3Aen&oc=11, https://news.google.com/rss/search?q=crypto+regulation+when:1d+source:(cointelegraph%7Creuters%7Cbloomberg)&hl=en-US&gl=US&ceid=US:en, https://news.google.com/rss/search?q=when:24h+allinurl:bloomberg.com&hl=en-US&gl=US&ceid=US%3Aen, https://protos.com/feed, https://s3.coinmarketcap.com/generated/rss/alexandria/articles.xml, https://smartliquidity.info/feed, https://thebitcoinnews.com/feed, https://thedefiant.io/api/feed, https://thedefiant.io/feed, https://time.com/nextadvisor/feed, https://tintucbitcoin.com/feed, https://u.today/rss, https://usethebitcoin.com/feed, https://vn.beincrypto.com/feed, https://www.ccn.com/analysis/crypto-analysis/feeds, https://www.ccn.com/news/crypto-news/feeds, https://www.chainalysis.com/blog/tag/chainalysis/feed, https://www.cnbc.com/id/10000664/device/rss/rss.html, https://www.coindesk.com/arc/outboundfeeds/rss, https://www.coindesk.com/arc/outboundfeeds/rss/?outputType=xml, https://www.coindeskjapan.com/feed, https://www.coolwallet.io/blogs/blog.atom, https://www.cryptobreaking.com/feed, https://www.elfinanciero.com.mx/rss, https://www.elliptic.co/blog/rss.xml, https://www.ft.com, https://www.newsbtc.com/feed, https://www.reddit.com/r/CryptoCurrency/.rss, https://www.theblock.co/rss.xml?feed=news.xml, https://www.tronweekly.com/feed, https://www.trustnodes.com/feed, https://zebpay.com/feed, https://zycrypto.com/feed" # @param {type:"string"}

# Chuyển chuỗi text thành list
RSS_LINKS = [link.strip() for link in RSS_LINKS_RAW.replace('\n', ',').split(',') if link.strip()]

In [ ]:
# @title 2. Định nghĩa Hệ thống phân tích

class TinPhanTich(BaseModel):
    index: int
    phu_hop: bool

def goi_gemini(noi_dung_prompt, api_key):
    # Cơ chế Retry khi gặp lỗi giới hạn (429)
    max_retries = 3
    for attempt in range(max_retries):
        try:
            client = genai.Client(api_key=api_key)
            config = types.GenerateContentConfig(
                response_mime_type="application/json",
                response_schema=list[TinPhanTich],
            )
            response = client.models.generate_content(
                model="gemini-2.5-flash",
                contents=noi_dung_prompt,
                config=config
            )
            return json.loads(response.text) if response.text else None
            
        except Exception as e:
            error_msg = str(e)
            if "429" in error_msg or "RESOURCE_EXHAUSTED" in error_msg:
                wait_time = 60 
                print(f"{YELLOW}⚠️ Gặp lỗi Quota (429). Đang nghỉ giải lao {wait_time} giây trước khi thử lại...{RESET}")
                time.sleep(wait_time)
            else:
                print(f"{RED}!!! Lỗi Gemini: {e}{RESET}")
                if attempt == max_retries - 1: return None
                time.sleep(5)
    return None

async def quet_rss_async(url, session, sem):
    """Tải và parse RSS bất đồng bộ với giới hạn luồng và cơ chế Retry cho lỗi DNS."""
    async with sem:
        max_retries = 2
        for i in range(max_retries + 1):
            try:
                # Staggered delay để tránh dồn dập DNS query
                await asyncio.sleep(random.uniform(0.5, 1.5))
                async with session.get(url, timeout=15, ssl=False) as response:
                    html = await response.text()
                    return feedparser.parse(html)
            except Exception as e:
                error_msg = str(e)
                # Nếu là lỗi DNS (gaierror), thử lại sau khi nghỉ
                if "gaierror" in error_msg or "nodename nor servname" in error_msg:
                    if i < max_retries:
                        await asyncio.sleep(2 * (i + 1))
                        continue
                
                if "Header value is too long" not in error_msg:
                    print(f"{RED}!!! Lỗi khi quét {url}: {error_msg[:100]}{RESET}")
                return None

async def lay_noi_dung_chi_tiet(url, trinh_duyet):
    """Bóc tách nội dung với logic Human-like và xử lý Redirect Google News"""
    trang = None
    ngu_canh = None
    try:
        thiet_bi = random.choice(THIET_BI_GIA_LAP)
        ngu_canh = await trinh_duyet.new_context(
            user_agent=thiet_bi['ua'],
            viewport=thiet_bi['vp'],
            is_mobile=thiet_bi['is_mobile']
        )
        trang = await ngu_canh.new_page()
        
        # Human-like delay
        await asyncio.sleep(random.uniform(1, 3))
        
        # Tăng timeout cho Google News
        timeout_val = 60000 if "news.google.com" in url else 45000
        response = await trang.goto(url, wait_until="domcontentloaded", timeout=timeout_val)
        
        # Xử lý Redirect Google News
        if "news.google.com" in url:
            await asyncio.sleep(random.uniform(3, 5))
            try:
                await trang.wait_for_load_state("networkidle", timeout=10000)
            except:
                pass
        
        if response and response.status == 200:
            # Human-like scroll
            for _ in range(random.randint(1, 3)):
                await trang.mouse.wheel(0, random.randint(300, 700))
                await asyncio.sleep(random.uniform(0.5, 1.2))
            
            content = await trang.evaluate("() => document.body.innerText")
            if content and len(content.strip()) > 500:
                return content.strip()
        
        return "Không thể bóc tách nội dung."
    except:
        return "Không thể bóc tách nội dung."
    finally:
        if trang: await trang.close()
        if ngu_canh: await ngu_canh.close()
def thong_ke_loi(tin_phu_hop):
    error_links = []
    for entry in tin_phu_hop:
        content = entry.get("noi_dung")
        if not content or content == "Không thể bóc tách nội dung." or content.strip() == "":
            error_links.append(entry.get("duong_dan"))
    
    # Lưu file link lỗi với timestamp
    timestamp = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
    file_error = f"error_links_{timestamp}.txt"
    with open(file_error, 'w', encoding='utf-8') as f:
        for link in error_links:
            f.write(f"{link}\n")
    print(f"{CYAN}--- Đã lưu danh sách link lỗi tại: {RESET}{YELLOW}{file_error}{RESET}")

    if error_links:
        print(f"\n{RED}{BOLD}⚠️ THỐNG KÊ LỖI BÓC TÁCH:{RESET}")
        print(f"Tổng số link lỗi: {len(error_links)} / {len(tin_phu_hop)}")
        from collections import Counter
        domains = [link.split('/')[2] if '://' in link else link for link in error_links]
        domain_counts = Counter(domains)
        for domain, count in domain_counts.items():
            print(f"  - {domain}: {count}")
        print(f"\n{YELLOW}Danh sách link lỗi (Xem chi tiết trong file {file_error}){RESET}")
    else:
        print(f"\n{GREEN}✅ Chúc mừng! Không có link nào bị lỗi bóc tách.{RESET}")

In [ ]:
# @title 3. Thực thi quy trình lọc tin

async def main():
    if GOOGLE_API_KEY == "NHAP_KEY_O_DAY":
        print(f"{RED}❌ Vui lòng nhập GOOGLE_API_KEY ở Bước 1 trước khi chạy!{RESET}")
        return
        
    print(f"{CYAN}{BOLD}🚀 Đang quét nhanh {len(RSS_LINKS)} nguồn RSS (Tuần tự)...{RESET}")
    kho_tin_tho = []
    tieu_de_da_lay = set()
    
    limit_sec = LIMIT_DAYS * 24 * 3600
    now = time.time()
    
    # Giới hạn 5 nguồn RSS tải cùng lúc để tránh nghẽn DNS/Block IP
    sem_rss = asyncio.Semaphore(1)
 # Chế độ tuần tự

    connector = aiohttp.TCPConnector(ssl=False)
    async with aiohttp.ClientSession(
        connector=connector, 
        headers={"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, Gecko) Chrome/119.0.0.0 Safari/537.36"}
    ) as session:
        for i, url in enumerate(RSS_LINKS):
            print(f"  [{i+1}/{len(RSS_LINKS)}] Quét nguồn: {url[:60]}...", end=" ", flush=True)
            feed = await quet_rss_async(url, session, sem_rss)

            if not feed:
                print(f"{RED}Thất bại{RESET}")
                continue

            so_tin_moi_nguon = 0
        
            for entry in feed.entries:
                tieu_de = entry.get('title', '').strip()
                if not tieu_de or tieu_de in tieu_de_da_lay: continue
                
                published = entry.get('published_parsed')
                if published and (now - time.mktime(published) > limit_sec): continue
                
                kho_tin_tho.append({
                    'tieu_de': tieu_de,
                    'mo_ta': entry.get('summary', '').strip(),
                    'duong_dan': entry.get('link', '')
                })
                tieu_de_da_lay.add(tieu_de)
                so_tin_moi_nguon += 1
    
            if so_tin_moi_nguon > 0:
                print(f"{GREEN}Thành công ({so_tin_moi_nguon} tin mới){RESET}")
            else:
                print(f"{YELLOW}Thành công (0 tin mới){RESET}")

    tong_so_tin = len(kho_tin_tho)
    print(f"{GREEN}✅ Đã lấy và lọc trùng: {tong_so_tin} tin Crypto mới.{RESET}")
    
    if not kho_tin_tho: return

    # --- LỌC AI ---
    BATCH_SIZE = 150
    tin_phu_hop = []
    print(f"\n{CYAN}{BOLD}🔄 AI (Gemini) đang lọc nội dung...{RESET}")
    
    for start_idx in range(0, tong_so_tin, BATCH_SIZE):
        end_idx = min(start_idx + BATCH_SIZE, tong_so_tin)
        batch_tin = kho_tin_tho[start_idx:end_idx]
        print(f"  > Check đợt {start_idx//BATCH_SIZE + 1} ({len(batch_tin)} tin)...")
        
        prompt = f"Ngữ cảnh phân tích: {SHARK_CONTEXT}\n\n"
        prompt += f"Hãy lọc ra các tin tức THỰC SỰ liên quan đến thị trường CRYPTO trong danh sách này:\n"
        for i, t in enumerate(batch_tin):
            prompt += f"- Bài {i}: {t['tieu_de']}\n"

        results_ai = goi_gemini(prompt, GOOGLE_API_KEY)
        if results_ai:
            for r in results_ai:
                if r.get('phu_hop'):
                    local_idx = r.get('index')
                    if local_idx is not None and local_idx < len(batch_tin):
                        tin_phu_hop.append(batch_tin[local_idx])
        
        if end_idx < tong_so_tin:
            time.sleep(2)

    if tin_phu_hop:
        # LƯU FILE TIN ĐÃ LỌC BỞI AI (Để phòng hờ)
        file_loc_ai = f"tin_loc_ai_{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.json"
        with open(file_loc_ai, "w", encoding="utf-8") as f:
            json.dump(tin_phu_hop, f, ensure_ascii=False, indent=2)
        print(f"{GREEN}--- Đã lưu danh sách tin phù hợp (Chưa bóc nội dung): {RESET}{YELLOW}{file_loc_ai}{RESET}")
        print(f"\n{CYAN}{BOLD}🎯 Bắt đầu bóc tách {len(tin_phu_hop)} tin phù hợp (Giả lập Người dùng + Redirect Fix)...{RESET}")
        limit = asyncio.Semaphore(CRAWL_CONCURRENCY)

        async def cào_tin_với_limit(index, tin_obj, browser_instance):
            async with limit:
                # Staggered delay
                await asyncio.sleep(index * random.uniform(0.3, 0.6))
                print(f"  {YELLOW}[{index+1}/{len(tin_phu_hop)}] --- Đang bóc:{RESET} {tin_obj['tieu_de'][:60]}...")
                tin_obj['noi_dung'] = await lay_noi_dung_chi_tiet(tin_obj['duong_dan'], browser_instance)

        async with async_playwright() as p:
            browser = await p.chromium.launch(headless=True)
            # ctx context removed
            tasks = [cào_tin_với_limit(i, t, browser) for i, t in enumerate(tin_phu_hop)]
            await asyncio.gather(*tasks)
            await browser.close()
            
        # FILE KẾT QUẢ CUỐI CÙNG
        filename = f"bao_cao_crypto_{datetime.now().strftime('%Y-%m-%d')}.txt"
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(tin_phu_hop, f, ensure_ascii=False, indent=2)
            
        print(f"\n{GREEN}{BOLD}🎉 HOÀN TẤT! Đã lưu báo cáo tại: {filename}{RESET}")
        thong_ke_loi(tin_phu_hop)
        
        print(f"\n{CYAN}📩 Đang chuẩn bị tải file kết quả cuối cùng về máy...{RESET}")
        try:
            from google.colab import files
            files.download(filename)
        except:
            pass
    else:
        print(f"\n{RED}❌ Không tìm thấy tin nào phù hợp.{RESET}")

await main()